# LLM-Based Entity Matching Quickstart

This notebook demonstrates how to use the `LLMBasedMatcher` for entity matching using large language models. We'll show both zero-shot and few-shot approaches using OpenAI's GPT-5 Nano model.

## Setup and Dependencies

First, let's check if the required dependencies are available and set up our environment.

In [1]:
import os
from typing import List
import pandas as pd
import tempfile
from pathlib import Path
from dotenv import load_dotenv
from PyDI.io import load_csv

load_dotenv()

# Check for LangChain OpenAI integration
try:
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import AIMessage, BaseMessage
    OPENAI_AVAILABLE = True
    print("✅ LangChain OpenAI integration available")
except ImportError:
    OPENAI_AVAILABLE = False
    print("❌ LangChain OpenAI not available. Install with: pip install langchain-openai")


try:
    from PyDI.entitymatching import LLMBasedMatcher
    print("✅ PyDI LLMBasedMatcher available")
except ImportError:
    print("❌ PyDI LLMBasedMatcher not available. Make sure PyDI is properly installed.")

✅ LangChain OpenAI integration available
✅ PyDI LLMBasedMatcher available


In [2]:
# Import additional modules for data loading
from PyDI.io import load_xml
from PyDI.entitymatching import ensure_record_ids
import numpy as np

def repo_root():
    """Return the repository root directory."""
    # For notebooks in PyDI/examples/, go up 2 levels to reach repo root
    if '__file__' in globals():
        return Path(__file__).parent.parent.parent
    else:
        # In Jupyter, find the pyproject.toml to locate repo root
        current = Path.cwd()
        while current != current.parent:
            if (current / 'pyproject.toml').exists():
                return current
            current = current.parent
        return Path.cwd()  # fallback

# Load movie datasets
root = repo_root()
academy_path = root / "input" / "movies" / "entitymatching" / "data" / "academy_awards.xml"
actors_path = root / "input" / "movies" / "entitymatching" / "data" / "actors.xml"

print(f"Academy awards data: {academy_path}")
print(f"Actors data: {actors_path}")

# Load datasets using PyDI's provenance-aware XML loader
academy_df = load_xml(academy_path, name="academy_awards")
actors_df = load_xml(actors_path, name="actors")

# Ensure datasets have record IDs for entity matching
academy_df = ensure_record_ids(academy_df)
actors_df = ensure_record_ids(actors_df)

print(f"\nAcademy Awards shape: {academy_df.shape}")
print(f"Academy Awards columns: {list(academy_df.columns)}")
print(f"Sample Academy IDs: {academy_df['_id'].head(3).tolist()}")

print(f"\nActors shape: {actors_df.shape}")
print(f"Actors columns: {list(actors_df.columns)}")
print(f"Sample Actors IDs: {actors_df['_id'].head(3).tolist()}")

print("\n=== Academy Awards Dataset Sample ===")
display(academy_df[['_id', 'title', 'actor_name', 'date', 'director_name']].head(3))

print("\n=== Actors Dataset Sample ===") 
display(actors_df[['_id', 'title', 'actor_name', 'date', 'actors_actor_birthday']].head(3))

Academy awards data: /Users/aaronsteiner/Documents/GitHub/PyDI/input/movies/entitymatching/data/academy_awards.xml
Actors data: /Users/aaronsteiner/Documents/GitHub/PyDI/input/movies/entitymatching/data/actors.xml

Academy Awards shape: (4592, 8)
Academy Awards columns: ['academy_awards_id', 'id', 'title', 'actor_name', 'date', 'director_name', 'oscar', '_id']
Sample Academy IDs: ['academy_awards_000000', 'academy_awards_000001', 'academy_awards_000002']

Actors shape: (149, 8)
Actors columns: ['actors_id', 'id', 'title', 'actor_name', 'actors_actor_birthday', 'actors_actor_birthplace', 'date', '_id']
Sample Actors IDs: ['actors_000000', 'actors_000001', 'actors_000002']

=== Academy Awards Dataset Sample ===


,_id,title,actor_name,date,director_name
0,academy_awards_000000,Biutiful,Javier Bardem,2010-01-01,NaN
1,academy_awards_000001,True Grit,Jeff Bridges,2010-01-01,Joel Coen
2,academy_awards_000002,True Grit,Jeff Bridges,2010-01-01,Ethan Coen



=== Actors Dataset Sample ===


,_id,title,actor_name,date,actors_actor_birthday
0,actors_000000,7th Heaven,Janet Gaynor,1929-01-01,1906-01-01
1,actors_000001,Coquette,Mary Pickford,1930-01-01,1892-01-01
2,actors_000002,The Divorcee,Norma Shearer,1931-01-01,1902-01-01


In [3]:
test_path = root / "input" / "movies" / "entitymatching" / "splits" / "gs_academy_awards_2_actors_test.csv"

blocking_gs = pd.read_csv(test_path, names=["id1", "id2", "label"])
blocking_gs

,id1,id2,label
0,academy_awards_4529,actors_2,True
1,academy_awards_4500,actors_3,True
2,academy_awards_4475,actors_4,True
3,academy_awards_4446,actors_5,True
4,academy_awards_4399,actors_6,True
...,...,...,...
3342,academy_awards_3765,actors_15,False
3343,academy_awards_1049,actors_65,False
3344,academy_awards_1115,actors_101,False
3345,academy_awards_3244,actors_101,True


In [4]:
def _id_to_pydi_id(id, df):
    return df[df["id"] == id]["_id"].values[0]


In [5]:
# get 20 positive and 20 negative examples
positive_examples = blocking_gs[blocking_gs["label"] == 1].sample(20)
negative_examples = blocking_gs[blocking_gs["label"] == 0].sample(20)

gs_examples = pd.concat([positive_examples, negative_examples])

# create candidate pairs from gold standard
candidates = gs_examples.apply(lambda row: (row["id1"], row["id2"]), axis=1).tolist()

# update these ids with the _id used in the datasets
candidates = [(_id_to_pydi_id(id1, academy_df), _id_to_pydi_id(id2, actors_df)) for id1, id2 in candidates]

print(candidates[:5])

[('academy_awards_004279', 'actors_000008'), ('academy_awards_004451', 'actors_000083'), ('academy_awards_001547', 'actors_000055'), ('academy_awards_001887', 'actors_000048'), ('academy_awards_002633', 'actors_000033')]


In [6]:
# Initialize OpenAI GPT-5 Nano
chat_model = ChatOpenAI(
    model="gpt-5-nano",  
    max_tokens=500,        
    #temperature=0.0,       
    reasoning_effort="minimal",  
)

print(f"✅ Configured {chat_model.model_name} with temperature={chat_model.temperature}")

# Test the model with a movie matching prompt
test_response = chat_model.invoke("""Are the movies 'Casablanca' (1942) and 'Casablanca' (1943) the same? Analyze the provided records carefully and return your decision as strict JSON in this format:
{{"match": true|false, "score": <float between 0.0 and 1.0>, "explanation": "<brief explanation>"}}

Guidelines:
- score should reflect your confidence (1.0 = definitely same entity, 0.0 = definitely different)
- match should be true if score >= 0.5, false otherwise  
- explanation should be concise (1-2 sentences)
- Consider variations in naming, formatting, abbreviations, and data quality
- Respond with ONLY the JSON object and nothing else.""")
print(f"Model test response: {test_response.content}")

✅ Configured gpt-5-nano with temperature=None
Model test response: {"match": true, "score": 0.78, "explanation": "Both records refer to the well-known classic film Casablanca, with the same title; differences in year (1942 vs 1943) could reflect production vs release records, but pertain to the same work."}


In [7]:
# Create the LLM matcher
matcher = LLMBasedMatcher()

# Perform zero-shot matching on movie data
print("🎬 Running zero-shot movie entity matching...")
if OPENAI_AVAILABLE and os.getenv('OPENAI_API_KEY'):
    print("   This may take a few moments as we call the OpenAI API...")

matches_zero_shot = matcher.match(
    academy_df, actors_df, candidates,
    chat_model=chat_model,
    fields=["title", "actor_name", "date"],  # Key movie attributes
    threshold=0.7,  
    debug=True  
)

print("Zero-shot matching results:")
display(matches_zero_shot)

print(f"\nFound {len(matches_zero_shot)} matches out of {len(candidates)} candidates")
if len(matches_zero_shot) > 0:
    print(f"Average confidence score: {matches_zero_shot['score'].mean():.3f}")
    print("\nTop movie matches:")
    for _, match in matches_zero_shot.sort_values('score', ascending=False).head(3).iterrows():
        left_rec = academy_df[academy_df['_id'] == match['id1']].iloc[0]
        right_rec = actors_df[actors_df['_id'] == match['id2']].iloc[0]
        print(f"  Score {match['score']:.3f}: '{left_rec['title']}' ({left_rec['date'][:4]}) ↔ '{right_rec['title']}' ({right_rec['date'][:4]})")
        if pd.notna(left_rec.get('actor_name')) and pd.notna(right_rec.get('actor_name')):
            print(f"    Actors: {left_rec['actor_name']} vs {right_rec['actor_name']}")
        if match['notes']:
            print(f"    → {match['notes']}")
else:
    print("No matches found above the threshold.")
    print("This is normal with mock data - try lowering the threshold to see more results.")

🎬 Running zero-shot movie entity matching...
   This may take a few moments as we call the OpenAI API...
Zero-shot matching results:


,id1,id2,score,notes
0,academy_awards_004451,actors_000083,0.72,Both records refer to The Champ starring Walla...
1,academy_awards_001547,actors_000055,0.72,Both records refer to the same film 'Sophie’s ...
2,academy_awards_002633,actors_000033,0.75,Both records refer to the same film 'Two Women...
3,academy_awards_001397,actors_000058,0.72,Titles are the same work with minor wording di...
4,academy_awards_002002,actors_000046,0.72,Same title and actor name with only one-year d...
5,academy_awards_000914,actors_000067,0.72,"Both records refer to the same film/title ""Blu..."
6,academy_awards_000612,actors_000143,0.75,Both records refer to the same film 'American ...
7,academy_awards_004484,actors_000003,0.72,Both records refer to the same film 'Min and B...
8,academy_awards_004509,actors_000002,0.72,Same title and actor; dates differ likely due ...
9,academy_awards_003821,actors_000092,0.75,Both records refer to the film Yankee Doodle D...



Found 12 matches out of 40 candidates
Average confidence score: 0.732

Top movie matches:
  Score 0.750: 'Two Women' (1961) ↔ 'Two Women' (1962)
    Actors: Sophia Loren vs Sophia Loren
    → Both records refer to the same film 'Two Women' starring Sophia Loren; dates differ by year (likely release vs award year) but actor and title align.
  Score 0.750: 'American Beauty' (1999) ↔ 'American Beauty' (2000)
    Actors: Kevin Spacey vs Kevin Spacey
    → Both records refer to the same film 'American Beauty' and the same actor Kevin Spacey; date differs (award year vs release year) but likely the same entity in different contexts.
  Score 0.750: 'Yankee Doodle Dandy' (1942) ↔ 'Yankee Doodle Dandy' (1943)
    Actors: James Cagney vs James Cagney
    → Both records refer to the film Yankee Doodle Dandy starring James Cagney; date differs by year potentially due to release vs award date, but core entity matches.


In [8]:
from PyDI.entitymatching import EntityMatchingEvaluator

# Convert blocking gold standard IDs to PyDI `_id` format
_eval_pairs = blocking_gs.copy()
_eval_pairs['id1'] = _eval_pairs['id1'].apply(lambda x: _id_to_pydi_id(x, academy_df))
_eval_pairs['id2'] = _eval_pairs['id2'].apply(lambda x: _id_to_pydi_id(x, actors_df))
if 'label' in _eval_pairs.columns:
    _eval_pairs['label'] = _eval_pairs['label'].map({True: 1, 'TRUE': 1, False: 0, 'FALSE': 0}).astype(int)

# Evaluate zero-shot results
evaluation = EntityMatchingEvaluator.evaluate(
    corr=matches_zero_shot,
    test_pairs=_eval_pairs,
    threshold=0.7,
    candidate_pairs=pd.DataFrame(candidates, columns=['id1', 'id2']),
    total_possible_pairs=len(academy_df) * len(actors_df),
    out_dir=str(root / 'PyDI' / 'examples' / 'output' / 'entitymatching' / 'llm')
)

print("\nEvaluation metrics (zero-shot):")
for key in ["precision", "recall", "f1", "accuracy", "true_positives", "false_positives", "false_negatives"]:
    if key in evaluation and evaluation[key] is not None:
        val = evaluation[key]
        print(f"  {key}: {val:.3f}" if isinstance(val, float) else f"  {key}: {val}")


Evaluation metrics (zero-shot):
  precision: 1.000
  recall: 0.255
  f1: 0.407
  accuracy: 0.990
  true_positives: 12
  false_positives: 0
  false_negatives: 35


In [13]:
# Define few-shot examples using movie domain knowledge
few_shot_examples = [
    # Positive example - same movie with slight variations
    (
        {"title": "Casablanca", "actor_name": "Humphrey Bogart", "date": "1942-01-01"},
        {"title": "Casablanca", "actor_name": "Humphrey Bogart", "date": "1942-01-01"},
        '{"match": true, "score": 1.0, "explanation": "Exact match - same movie, actor, and year"}'
    ),
    # Positive example - same movie, different actors mentioned
    (
        {"title": "The Godfather", "actor_name": "Marlon Brando", "date": "1972-01-01"},
        {"title": "The Godfather", "actor_name": "Al Pacino", "date": "1972-01-01"},
        '{"match": true, "score": 0.9, "explanation": "Same movie and year, different actors from same cast"}'
    ),
    # Negative example - different movies with similar themes
    (
        {"title": "The Dark Knight", "actor_name": "Christian Bale", "date": "2008-01-01"},
        {"title": "Batman Begins", "actor_name": "Christian Bale", "date": "2005-01-01"},
        '{"match": false, "score": 0.2, "explanation": "Different Batman movies despite same actor"}'
    ),
    # Negative example - completely different movies
    (
        {"title": "Titanic", "actor_name": "Leonardo DiCaprio", "date": "1997-01-01"},
        {"title": "The Matrix", "actor_name": "Keanu Reeves", "date": "1999-01-01"},
        '{"match": false, "score": 0.0, "explanation": "Completely different movies, actors, and years"}'
    )
]

import json

print(f"Movie-specific few-shot examples ({len(few_shot_examples)} examples):")
for i, (left_ex, right_ex, expected_json) in enumerate(few_shot_examples, 1):
    print(f"\nExample {i}:")
    print(f"  Movie A: '{left_ex['title']}' ({left_ex['date'][:4]}) - {left_ex.get('actor_name', 'N/A')}")
    print(f"  Movie B: '{right_ex['title']}' ({right_ex['date'][:4]}) - {right_ex.get('actor_name', 'N/A')}")
    expected = json.loads(expected_json)
    print(f"  Expected: {'Match' if expected['match'] else 'No match'} (score: {expected['score']})")


# Perform few-shot matching
print(f"\n🎬 Running few-shot movie matching with {len(few_shot_examples)} examples...")
matches_few_shot = matcher.match(
    academy_df, actors_df, candidates,
    chat_model=chat_model,
    fields=["title", "actor_name", "date"],
    few_shots=few_shot_examples,  
)

print("Few-shot matching results:")
display(matches_few_shot)

print(f"\nFound {len(matches_few_shot)} matches out of {len(candidates)} candidates")
if len(matches_few_shot) > 0:
    print(f"Average confidence score: {matches_few_shot['score'].mean():.3f}")
    
    print("\nComparison with zero-shot results:")
    print(f"  Zero-shot matches: {len(matches_zero_shot)}")
    print(f"  Few-shot matches:  {len(matches_few_shot)}")
    
    if len(matches_zero_shot) > 0 and len(matches_few_shot) > 0:
        avg_zero = matches_zero_shot['score'].mean()
        avg_few = matches_few_shot['score'].mean()
        print(f"  Average scores: {avg_zero:.3f} → {avg_few:.3f} ({'↑' if avg_few > avg_zero else '↓' if avg_few < avg_zero else '='} {abs(avg_few - avg_zero):.3f})")
        
    print("\nTop few-shot matches:")
    for _, match in matches_few_shot.sort_values('score', ascending=False).head(3).iterrows():
        left_rec = academy_df[academy_df['_id'] == match['id1']].iloc[0] 
        right_rec = actors_df[actors_df['_id'] == match['id2']].iloc[0]
        print(f"  Score {match['score']:.3f}: '{left_rec['title']}' ↔ '{right_rec['title']}'")
        if match['notes']:
            print(f"    → {match['notes']}")
else:
    print("No matches found with current threshold.")
    print("Few-shot examples help the model understand movie matching patterns better.")

Movie-specific few-shot examples (4 examples):

Example 1:
  Movie A: 'Casablanca' (1942) - Humphrey Bogart
  Movie B: 'Casablanca' (1942) - Humphrey Bogart
  Expected: Match (score: 1.0)

Example 2:
  Movie A: 'The Godfather' (1972) - Marlon Brando
  Movie B: 'The Godfather' (1972) - Al Pacino
  Expected: Match (score: 0.9)

Example 3:
  Movie A: 'The Dark Knight' (2008) - Christian Bale
  Movie B: 'Batman Begins' (2005) - Christian Bale
  Expected: No match (score: 0.2)

Example 4:
  Movie A: 'Titanic' (1997) - Leonardo DiCaprio
  Movie B: 'The Matrix' (1999) - Keanu Reeves
  Expected: No match (score: 0.0)

🎬 Running few-shot movie matching with 4 examples...
Few-shot matching results:


,id1,id2,score,notes
0,academy_awards_004500,actors_000080,0.60,Same title and actor; dates differ (1929 vs 19...
1,academy_awards_000612,actors_000143,0.60,"Same title, actor, but different date; likely ..."
2,academy_awards_002852,actors_000029,0.60,"Same movie and actor, but different year (1957..."
3,academy_awards_003260,actors_000022,0.70,Same title and actor; date differs by one year...
4,academy_awards_002573,actors_000112,0.60,"Same movie and actor, but different year (1962..."
5,academy_awards_002740,actors_000109,0.70,"Same title and actor, most likely same film th..."
6,academy_awards_002002,actors_000046,0.60,Same title and actor; dates differ by one year...
7,academy_awards_004455,actors_000004,0.60,"Same title and actor, but different year indic..."
8,academy_awards_004539,actors_000001,0.60,"Same title and actor, dates differ (1928 vs 19..."
9,academy_awards_000507,actors_000145,0.95,"Same title, actor, and date; differing interna..."



Found 10 matches out of 40 candidates
Average confidence score: 0.655

Comparison with zero-shot results:
  Zero-shot matches: 13
  Few-shot matches:  10
  Average scores: 0.738 → 0.655 (↓ 0.083)

Top few-shot matches:
  Score 0.950: 'Training Day' ↔ 'Training Day'
    → Same title, actor, and date; differing internal IDs do not affect entity equivalence
  Score 0.700: 'Born Yesterday' ↔ 'Born Yesterday'
    → Same title and actor; date differs by one year which may reflect different release years in awards context
  Score 0.700: 'Ben-Hur' ↔ 'Ben-Hur'
    → Same title and actor, most likely same film though dates differ by one year (release date vs awards year); consider potential discrepancy but high likelihood of same entity


In [14]:
evaluation = EntityMatchingEvaluator.evaluate(
    corr=matches_few_shot,
    test_pairs=_eval_pairs,
    threshold=0.7,
    candidate_pairs=pd.DataFrame(candidates, columns=['id1', 'id2']),
    total_possible_pairs=len(academy_df) * len(actors_df),
    out_dir=str(root / 'PyDI' / 'examples' / 'output' / 'entitymatching' / 'llm')
)

print("\nEvaluation metrics (few-shot):")
for key in ["precision", "recall", "f1", "accuracy", "true_positives", "false_positives", "false_negatives"]:
    if key in evaluation and evaluation[key] is not None:
        val = evaluation[key]
        print(f"  {key}: {val:.3f}" if isinstance(val, float) else f"  {key}: {val}")


Evaluation metrics (few-shot):
  precision: 1.000
  recall: 0.064
  f1: 0.120
  accuracy: 0.987
  true_positives: 3
  false_positives: 0
  false_negatives: 44


In [17]:
# Custom prompt for movie matching
custom_prompt = """You are an expert in movie database entity resolution with deep knowledge of film history and industry practices.

Your task is to determine if two movie records refer to the same film. Consider these factors:
- Movies are often listed in different databases with slight title variations
- Release dates may vary by a year due to festival releases, production dates, or international releases
- The same movie may list different actors if records focus on different cast members
- Award databases may use official titles while others use popular titles
- Consider sequels, remakes, and reboots as different movies even with similar titles
- TV movies vs theatrical releases are typically different entities

Analyze the provided records carefully and return your decision as strict JSON in this format:
{{"match": true|false, "score": <float between 0.0 and 1.0>, "explanation": "<brief explanation>"}}

Be precise - focus on whether these records represent the same specific film production."""

print(f"🎬 Running matching with movie-specific custom prompt...")
matches_custom = matcher.match(
    academy_df, actors_df, candidates,
    chat_model=chat_model,
    system_prompt=custom_prompt,
    fields=["title", "actor_name", "date"],
    threshold=0.6,  
)

print("Custom prompt matching results:")
display(matches_custom)

print(f"\nFound {len(matches_custom)} matches out of {len(candidates)} candidates")
if len(matches_custom) > 0:
    print(f"Average confidence score: {matches_custom['score'].mean():.3f}")
    print("\nDetailed movie match analysis:")
    for _, match in matches_custom.sort_values('score', ascending=False).head(3).iterrows():
        left_rec = academy_df[academy_df['_id'] == match['id1']].iloc[0]
        right_rec = actors_df[actors_df['_id'] == match['id2']].iloc[0]
        print(f"\n  Match Score: {match['score']:.3f}")
        print(f"  Academy Awards: '{left_rec['title']}' ({left_rec['date'][:4]})")
        if pd.notna(left_rec.get('actor_name')):
            print(f"    Actor: {left_rec['actor_name']}")
        if pd.notna(left_rec.get('director_name')):  
            print(f"    Director: {left_rec['director_name']}")
        print(f"  Actors DB: '{right_rec['title']}' ({right_rec['date'][:4]})")  
        print(f"    Actor: {right_rec['actor_name']}")
        if match['notes']:
            print(f"    Reasoning: {match['notes']}")
else:
    print("The custom prompt helps the model understand movie-specific matching criteria.")
    print("With real data and an actual LLM, this would show more nuanced movie matching.")

🎬 Running matching with movie-specific custom prompt...
Custom prompt matching results:


,id1,id2,score,notes
0,academy_awards_000612,actors_000143,0.72,Both records refer to the same film 'American ...
1,academy_awards_002852,actors_000029,0.75,Both records refer to The Three Faces of Eve s...
2,academy_awards_002341,actors_000038,0.72,"Both records refer to the same film, 'Who's Af..."
3,academy_awards_003260,actors_000022,0.75,Both records refer to the same film title and ...
4,academy_awards_002627,actors_000111,0.72,Both records refer to the same film Judgment a...
5,academy_awards_001783,actors_000050,0.62,"Both records share the same film title, Annie ..."
6,academy_awards_002573,actors_000112,0.62,Both records refer to the same film title and ...
7,academy_awards_002633,actors_000033,0.62,Both records title 'Two Women' with Sophia Lor...
8,academy_awards_002740,actors_000109,0.64,Both records reference a film titled Ben-Hur w...
9,academy_awards_002002,actors_000046,0.72,Both records refer to a film titled 'A Touch o...



Found 20 matches out of 40 candidates
Average confidence score: 0.721

Detailed movie match analysis:

  Match Score: 0.920
  Academy Awards: 'Training Day' (2001)
    Actor: Denzel Washington
  Actors DB: 'Training Day' (2001)
    Actor: Denzel Washington
    Reasoning: Both records reference the same film Training Day (2001) starring Denzel Washington, with identical title, lead actor, and release year. Minor variations in database categories do not indicate a different film; it is the same production.

  Match Score: 0.850
  Academy Awards: 'You'll Find Out' (1940)
  Actors DB: 'Gone With the Wind' (1940)
    Actor: Vivien Leigh
    Reasoning: Titles differ: 'You'll Find Out' is a 1940 film not related to 'Gone With the Wind'. The dates are the same but the titles and likely production context differ. One record references a mystery/anthology or comedy short; the other is the classic epic 'Gone with the Wind' (1939/1940 release). Actor reference (Vivien Leigh) may appear in both da

In [18]:
evaluation = EntityMatchingEvaluator.evaluate(
    corr=matches_custom,
    test_pairs=_eval_pairs,
    threshold=0.7,
    candidate_pairs=pd.DataFrame(candidates, columns=['id1', 'id2']),
    total_possible_pairs=len(academy_df) * len(actors_df),
    out_dir=str(root / 'PyDI' / 'examples' / 'output' / 'entitymatching' / 'llm')
)

print("\nEvaluation metrics (custom prompt):")
for key in ["precision", "recall", "f1", "accuracy", "true_positives", "false_positives", "false_negatives"]:
    if key in evaluation and evaluation[key] is not None:
        val = evaluation[key]
        print(f"  {key}: {val:.3f}" if isinstance(val, float) else f"  {key}: {val}")


Evaluation metrics (custom prompt):
  precision: 0.643
  recall: 0.191
  f1: 0.295
  accuracy: 0.987
  true_positives: 9
  false_positives: 5
  false_negatives: 38
